# Profiling and Timing Code

In the process of developing code and creating data processing pipelines, there are often trade-offs you can make between various implementations.
Early in developing your algorithm, it can be counterproductive to worry about such things. As Donald Knuth famously quipped, "We should forget about small efficiencies, say about 97% of the time: premature optimization is the root of all evil."

Once you have your code working, it can be useful to dig into its efficiency a bit.
Sometimes it's useful to check the execution time of a given command or set of commands; other times it's useful to dig into a multiline process and determine where the bottleneck lies in some complicated series of operations.
IPython provides access to a wide array of functionality for this kind of timing and profiling of code.
Here we'll discuss the following IPython magic commands:

- ``%time``: Time the execution of a single statement
- ``%%time``: Time the execution of an entire cell (must be first line)
- ``%timeit``: Time repeated execution of a single statement for more accuracy
- ``%%timeit``: Time repeadted execution of an entire cell (must be first line)
- ``%prun``: Run code with the profiler


## Timing Code Snippets: ``%timeit`` and ``%time``

``%timeit`` line-magic and ``%%timeit`` cell-magic can be used to time the repeated execution of snippets of code:

In [1]:
%timeit sum(range(100))

447 ns ± 117 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


Note that because this operation is so fast, ``%timeit`` automatically does a large number of repetitions.
For slower commands, ``%timeit`` will automatically adjust and perform fewer repetitions:

In [2]:
%%timeit
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

88.5 ms ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Sometimes repeating an operation is not the best option.
For example, if we have a list that we'd like to sort, we might be misled by a repeated operation.
Sorting a pre-sorted list is much faster (for many sorting algorithms) than sorting an unsorted list, so the repetition will skew the result:

In [3]:
import random
L = [random.random() for i in range(100000)]
%timeit L.sort()

192 μs ± 199 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


For this, the ``%time`` magic function may be a better choice. It also is a good choice for longer-running commands, when short, system-related delays are unlikely to affect the result.
Let's time the sorting of an unsorted and a presorted list:

In [4]:
import random
L = [random.random() for i in range(100000)]
print("sorting an unsorted list:")
%time L.sort()

sorting an unsorted list:
CPU times: user 15.6 ms, sys: 195 μs, total: 15.8 ms
Wall time: 15.7 ms


In [5]:
print("sorting an already sorted list:")
%time L.sort()

sorting an already sorted list:
CPU times: user 4.35 ms, sys: 16 μs, total: 4.36 ms
Wall time: 4.45 ms


Notice how much faster the presorted list is to sort, but notice also how much longer the timing takes with ``%time`` versus ``%timeit``, even for the presorted list!
This is a result of the fact that ``%timeit`` does some clever things under the hood to prevent system calls from interfering with the timing.
For example, it prevents cleanup of unused Python objects (known as *garbage collection*) which might otherwise affect the timing.
For this reason, ``%timeit`` results are usually noticeably faster than ``%time`` results.

For ``%time`` as with ``%timeit``, using the double-percent-sign cell magic syntax allows timing of multiline scripts:

In [6]:
%%time
total = 0
for i in range(1000):
    for j in range(1000):
        total += i * (-1) ** j

CPU times: user 140 ms, sys: 2.46 ms, total: 143 ms
Wall time: 141 ms


For more information on ``%time`` and ``%timeit``, as well as their available options, use the IPython help functionality (i.e., type ``%time?`` at the IPython prompt).

# Using time module

I often find myself wanting to benchmark chunks of larger functions. Here, the ``time`` module can be very helpful!

In [21]:
import time
def some_function():
    time.sleep(3) #this pauses execution - here it represents a placeholder for some larger chunk of code

    #section I care about benchmarking
    start_time = time.time() #timestamp when this section starts
    input("Please type some letters: ")
    stop_time = time.time() #timestamp when the section ends

    total_time = stop_time - start_time
    print(f"The total time to get user input was {total_time} seconds")

some_function()
    

Please type some letters:  a


The total time to get user input was 1.0414409637451172 seconds


# Memory Profiling
In many cases it is useful to know how much storage space something requires.  Python's system library (sys) provides a getsizeof() method that returns the number of bytes used by the variable.  

In [4]:
import sys
my_int = 1
my_string = "hello"
my_double = 3.14159


print(sys.getsizeof(my_int))
print(sys.getsizeof(my_string))
print(sys.getsizeof(my_double))

28
46
24


In the following cell, add at least 4 more new objects test their sizes.  See if you can find the primitive that takes the least space.  Do big numbers require more storage?  How is python different than Java when it comes to primitive storage sizes?

In [3]:
my_list = [1, 2, 3, 4, 5]
my_dict = {'a': 1, 'b': 2, 'c': 3}
my_bool = True
my_tuple = (1, 2, 3)
my_big_int = 1234567890123456789012345678901234567890

print(sys.getsizeof(my_list))
print(sys.getsizeof(my_dict))
print(sys.getsizeof(my_bool))
print(sys.getsizeof(my_tuple))
print(sys.getsizeof(my_big_int))

104
184
28
64
44


In the next cell, we explore what size differences exist between various lists and other data structures.  Add a data structure for a dict as well to see what size it is.  Why are some of these are equal to each other?  Reading the documentation might be helpful: https://docs.python.org/3/library/sys.html

In [8]:
print(sys.getsizeof([]))
print(sys.getsizeof([1]))
print(sys.getsizeof(['a quick brown fox jumped over the lazy dog']))
print(sys.getsizeof([1,2,3,4,5,6,7,8,9]))
print(sys.getsizeof(["a","quick","brown","fox","jumped","over","the","lazy","dog"]))
print(sys.getsizeof({'a': 1, 'b': 2, 'c': 3}))


56
64
64
136
136
184


If you haven't figured it out yet, the memory that is reported only accounts for the space to store the reference to the object, not to the underlying objects contained therein.  Therefore, we need a recursive way to find the actual memory footprint of the object.  Consider the following method...

In [11]:
from sys import getsizeof,stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}
                    
    Source: https://code.activestate.com/recipes/577504/
    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)


In the following cell, use the method above to calculate the actual size of the objects you created above.  Please record any reactions to this exercise in the last cell.  

In [14]:
print(total_size(my_big_int))
print(total_size(my_dict))
print(total_size(my_bool))
print(total_size(my_tuple))

44
394
28
148


# Put reactions to this assignment here

No reactions of note, I was already aware that objects are just the pointers to the underlying structures.

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

This content has been modified by Dr. Derek Riley